In [20]:
import numpy as np
from scipy.spatial.transform import Rotation as R


In [21]:
grid_res = 0.05
grid_params = {
    "x": (-0.1, 0.1),
#    "x": (-0.2, 0.2),
#    "x": (-0.4, 0.4),
#    "y": (-0.02, 0.30),
    "y": (-0.20, 0.30),
#    "y": (-0.40, 0.60),
    "theta": (-0.2, 0.2),
#    "num_x": 11,
#    "num_x": 22,
#    "num_x": 44,
#    "num_y": 17,
#    "num_y" : 26,
#    "num_y" : 52,
#    "num_t": 41,
    "num_t": 61,
    
}

grid_params['num_x'] = int((grid_params['x'][1] - grid_params['x'][0])/grid_res + 1)
grid_params['num_y'] = int((grid_params['y'][1] - grid_params['y'][0])/grid_res + 1)

In [22]:
def rotate(points, theta):
    theta -= np.pi / 2 #WHY?
    r = R.from_quat([0, 0, np.sin(theta/2), np.cos(theta/2)])
    points_rotated=r.apply(points)
    return points_rotated

# TODO: Fix rotation matrix and move ?~@/2 out here
def transform_local_to_world(pose, points):
    """Transform local coordinates to world coordinates"""
    x, y, z, theta = pose
    points_rotated = rotate(points,theta) #TODO: verify
    return points_rotated + np.array([x, y, z])

In [83]:
# TODO: Generate grid and rotate it later - don't regenerate it all the time
def grid_space(center, x, y, theta, num_x=9, num_y=15, num_t=9):
    """center is the global pose of the grid. x, y, theta are 2-tuples of (min, max)"""
    search_space = np.array(
        np.meshgrid(
            np.linspace(*x, num=num_x),
            np.linspace(*y, num=num_y),
            0,
            np.linspace(*theta, num=num_t),
        )
    ).T.reshape(-1, 4)
    print(search_space)
    print(search_space.shape)
    search_space[:, :3] = transform_local_to_world(center, search_space[:, :3])
    search_space[:, 3] += center[3]

    return search_space

In [86]:
search_space = grid_space(center=np.array([100,200,10, np.pi/2]), **grid_params)


[[-0.1  -0.2   0.   -0.2 ]
 [-0.1  -0.15  0.   -0.2 ]
 [-0.1  -0.1   0.   -0.2 ]
 ...
 [ 0.1   0.2   0.    0.2 ]
 [ 0.1   0.25  0.    0.2 ]
 [ 0.1   0.3   0.    0.2 ]]
(3355, 4)


In [87]:
print(search_space)

[[ 99.9        199.8         10.           1.37079633]
 [ 99.9        199.85        10.           1.37079633]
 [ 99.9        199.9         10.           1.37079633]
 ...
 [100.1        200.2         10.           1.77079633]
 [100.1        200.25        10.           1.77079633]
 [100.1        200.3         10.           1.77079633]]
